TODO:
- zet annotation_info dict om in een echte Class
- encapsulate begin_anchor en end_anchor, zorg dat ze verschillend kunnen worden geimplementeerd
- voeg get_begin_identifier() en get_end_identifier() methods toe, voor uniform client gebruik
- maak mijn gebruik van begin_char_offset en end_char_offset consistent met 1. TEI casus 2. Rik's begin en end voor attendants

In [1]:
import sys
sys.path.append('../packages')

import requests
import json
import uuid
from textservice import segmentedtext

Retrieve the annotation for a specific resolution in the sample 1728 data.

In [2]:
session_id = "meeting-1728-06-01-session-1"
url = "http://localhost:5001/annotations/"+session_id
response = requests.get(url)

In [3]:
response.json()

{'annotations': {'begin_anchor': 49716,
  'end_anchor': 50692,
  'id': 'meeting-1728-06-01-session-1',
  'image_coords': None,
  'label': 'sessions',
  'president': 're Van Borselen.',
  'resource_id': 'volume-1728',
  'session_date': '1728-06-01',
  'session_weekday': 'Martis',
  'session_year': 1728}}

In [4]:
session_annotation = response.json()['annotations']
session_annotation

{'begin_anchor': 49716,
 'end_anchor': 50692,
 'id': 'meeting-1728-06-01-session-1',
 'image_coords': None,
 'label': 'sessions',
 'president': 're Van Borselen.',
 'resource_id': 'volume-1728',
 'session_date': '1728-06-01',
 'session_weekday': 'Martis',
 'session_year': 1728}

In [5]:
resource_id = session_annotation['resource_id']
begin = session_annotation['begin_anchor']
end = session_annotation['end_anchor']

response = requests.get(f"http://localhost:5000/{resource_id}/segmentedtext/textgrid/{begin},{end}")

In [6]:
response.json()

{'textgrid': {'_ordered_segments': ['Martis den 1. Junii',
   '1728.',
   'P R /E S I D E,',
   'Den Heere Van Borselen.',
   'P R JE S E N T I BU S',
   'De Heeren Van Singendonck , van Heucke-',
   'lom , Coulman , van Heeckeren tot den',
   'Brandt zenborgh , Potgieter, met een enx-',
   'traordinaris Gedeputeerde uyt de Provin-',
   'cie van Gelderlandt.',
   'Vander Dussen , Bors van Waveren, Spie-',
   'ringh.',
   'Bout Noey , van Hoorn.',
   '7 an Renswoude.',
   'Van Glinstra , van Schwartzenbergh , van',
   'Goslinca, Banga.',
   'Van Rechteren, van felmuden , Ten Brincke;',
   'Scri verins.',
   'Van Tamminga.',
   'FE Resolutien , gisteren geno-',
   'D men, zyn gelelen en geresumeert ,',
   '7',
   'gelyeck oock geresumeert en gear-',
   'resteert zyn de Depesches daar uyt resul-',
   'roerende.',
   'u Ntfangen een Mifsive van den Resi-',
   'J dent Rumpf ‚ geschreven te Dresden',
   'den ses en twintighsten der voorlede',
   'e',
   'maandt, houdende advertentie.',
   'W

toon text_grid als een pseudo-visuele weergave, met segment index voorafgaand aan ieder line segment

In [7]:
textgrid = segmentedtext.IndexedSegmentedText.from_json(response.json()['textgrid'])
segments = textgrid._ordered_segments

for i, segment in enumerate(segments):
    print(f"{i+textgrid.text_grid_spec['begin_offset_in_resource']}: {segment}")

49716: Martis den 1. Junii
49717: 1728.
49718: P R /E S I D E,
49719: Den Heere Van Borselen.
49720: P R JE S E N T I BU S
49721: De Heeren Van Singendonck , van Heucke-
49722: lom , Coulman , van Heeckeren tot den
49723: Brandt zenborgh , Potgieter, met een enx-
49724: traordinaris Gedeputeerde uyt de Provin-
49725: cie van Gelderlandt.
49726: Vander Dussen , Bors van Waveren, Spie-
49727: ringh.
49728: Bout Noey , van Hoorn.
49729: 7 an Renswoude.
49730: Van Glinstra , van Schwartzenbergh , van
49731: Goslinca, Banga.
49732: Van Rechteren, van felmuden , Ten Brincke;
49733: Scri verins.
49734: Van Tamminga.
49735: FE Resolutien , gisteren geno-
49736: D men, zyn gelelen en geresumeert ,
49737: 7
49738: gelyeck oock geresumeert en gear-
49739: resteert zyn de Depesches daar uyt resul-
49740: roerende.
49741: u Ntfangen een Mifsive van den Resi-
49742: J dent Rumpf ‚ geschreven te Dresden
49743: den ses en twintighsten der voorlede
49744: e
49745: maandt, houdende advertentie.
49746: W

Simuleer selectie van een stukje tekst, nl 'Vrank-ryck' op regels 43 en 44

In [8]:
begin_anchor_id = 43
end_anchor_id = 44
begin_char_offset = 26
end_char_offset = 4

print(segments[43:45])

['bassadeur aan het Hof van Vranck-', 'ryck, geschreven te Parys den aght en twin-']


In [9]:
# print(segments[43][begin_offset:]+segments[44][:end_offset])
print(segments[begin_anchor_id][begin_char_offset:], end='')
for seg in segments[begin_anchor_id+1:end_anchor_id]:
    print(seg, end='')
print(segments[end_anchor_id][:end_char_offset])

Vranck-ryck


In [10]:
# find begin and end anchors by id
offset = textgrid.text_grid_spec['begin_offset_in_resource']

print(f'ba: {begin_anchor_id+offset} ea: {end_anchor_id+offset}')

ba: 49759 ea: 49760


In [11]:
entity_ann = {'resource_id': resource_id, 'label':'entity',\
              'begin_anchor': begin_anchor_id + offset,\
              'end_anchor': end_anchor_id + offset,\
              'id': 'annot_'+str(uuid.uuid4()), 'entity_type': 'loc', \
              'entity_text': 'Vrankryck'}

entity_ann

{'resource_id': 'volume-1728',
 'label': 'entity',
 'begin_anchor': 49759,
 'end_anchor': 49760,
 'id': 'annot_836c26a1-a293-46a0-bf4b-35e7928008e1',
 'entity_type': 'loc',
 'entity_text': 'Vrankryck'}

In [12]:
entity_ann['begin_char_offset'] = begin_char_offset
entity_ann['end_char_offset'] = end_char_offset

entity_ann

{'resource_id': 'volume-1728',
 'label': 'entity',
 'begin_anchor': 49759,
 'end_anchor': 49760,
 'id': 'annot_836c26a1-a293-46a0-bf4b-35e7928008e1',
 'entity_type': 'loc',
 'entity_text': 'Vrankryck',
 'begin_char_offset': 26,
 'end_char_offset': 4}

Sla de nieuwe annotatie op in annotationstore

In [13]:
headers = {"Content-Type": "application/json"}
url = "http://localhost:5001/annotations"
response = requests.put(url, data=json.dumps(entity_ann, indent=4, cls=segmentedtext.SegmentEncoder), headers=headers)

response

<Response [200]>

TODO NEXT: as a test retrieve annotations overlapping with meeting-1728-06-01-session-1 and check if the newly created entity is included

In [14]:
url = "http://localhost:5001/volume-1728/annotations/49716,50692"
response = requests.get(url)

In [15]:
response.json()

{'annotations': [{'begin_anchor': 49716,
   'end_anchor': 49716,
   'id': 'NL-HaNA_1.01.02_3783_0258-page-515-column-1-tr-0-line-0',
   'image_coords': {'bottom': 528,
    'height': 59,
    'left': 3653,
    'right': 4155,
    'top': 469,
    'width': 502},
   'label': 'lines',
   'resource_id': 'volume-1728'},
  {'begin_anchor': 49716,
   'end_anchor': 49716,
   'id': 'NL-HaNA_1.01.02_3783_0258-page-515-column-1-tr-0-line-0',
   'image_coords': {'bottom': 528,
    'height': 59,
    'left': 3653,
    'right': 4155,
    'top': 469,
    'width': 502},
   'label': 'textregions',
   'resource_id': 'volume-1728'},
  {'begin_anchor': 49717,
   'end_anchor': 49717,
   'id': 'NL-HaNA_1.01.02_3783_0258-page-515-column-1-tr-0-line-1',
   'image_coords': {'bottom': 582,
    'height': 56,
    'left': 3831,
    'right': 3962,
    'top': 526,
    'width': 131},
   'label': 'lines',
   'resource_id': 'volume-1728'},
  {'begin_anchor': 49717,
   'end_anchor': 49717,
   'id': 'NL-HaNA_1.01.02_3783_0258